In [ ]:
from simphony.libraries import sipann, siepic
from simphony.simulators import SweepSimulator

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

Define ring factory function

In [ ]:
def ring_factory(radius):
	halfring1 = sipann.HalfRing(500e-9, 220e-9, radius, 100e-9)
	halfring2 = sipann.HalfRing(500e-9, 220e-9, radius, 100e-9)
	terminator = siepic.Terminator()

	halfring1.rename_pins("pass", "midb", "in", "midt")
	halfring2.rename_pins("out", "midt", "term", "midb")

	halfring1.interface(halfring2)
	halfring2["term"].connect(terminator)

	return halfring1.circuit.to_subcircuit()

Define add-drop circuit

In [ ]:
wg_input = siepic.Waveguide(100e-6)
wg_out = siepic.Waveguide(100e-6)
terminator1 = siepic.Terminator()
terminator2 = siepic.Terminator()
ring1 = ring_factory(6e-6)
ring2 = ring_factory(7e-6)

ring1["in"].connect(wg_input["pin2"])
ring1["pass"].connect(terminator1)
ring1["out"].connect(ring2["in"])
ring2["pass"].connect(terminator2)
ring2["out"].connect(wg_out)

Simulate

In [ ]:
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(1, 3)
ax = fig.add_subplot(gs[0, :2])

simulator = SweepSimulator(1500e-9, 1600e-9)
simulator.connect(wg_input)

simulator.multiconnect(None, wg_out)
wl, t = simulator.simulate()
ax.plot(wl*1e9, t, lw="0.7")

ax.set_ylabel("Fractional Optical Power")
ax.set_xlabel("Wavelength (nm)")

plt.show()